# CIFAR10 (precompute conv output)

https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop, Adam
import bcolz

Using Theano backend.


In [3]:
batch_size = 32
num_classes = 10
epochs = 25

### The data, shuffled and split between train and test sets

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


### Convert class vectors to binary class matrices

In [6]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

### Create model

In [7]:
model = Sequential()

In [8]:
model.add(ZeroPadding2D((1, 1), input_shape=x_train.shape[1:]))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 34, 34)     0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    896         zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 16, 16)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 32, 16, 16)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

### initiate RMSprop optimizer

In [10]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

### Let's train the model using RMSprop

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [13]:
x_train /= 255
x_test /= 255

In [14]:
model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_data=(x_test, y_test), shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 16s - loss: 1.8558 - acc: 0.3282 - val_loss: 1.5835 - val_acc: 0.4326
Epoch 2/25
50000/50000 [==============================] - 16s - loss: 1.5115 - acc: 0.4551 - val_loss: 1.3569 - val_acc: 0.5211
Epoch 3/25
50000/50000 [==============================] - 16s - loss: 1.3645 - acc: 0.5112 - val_loss: 1.3302 - val_acc: 0.5239
Epoch 4/25
50000/50000 [==============================] - 16s - loss: 1.2647 - acc: 0.5524 - val_loss: 1.1810 - val_acc: 0.5858
Epoch 5/25
50000/50000 [==============================] - 16s - loss: 1.1855 - acc: 0.5828 - val_loss: 1.0727 - val_acc: 0.6233
Epoch 6/25
50000/50000 [==============================] - 16s - loss: 1.1222 - acc: 0.6050 - val_loss: 1.0654 - val_acc: 0.6269
Epoch 7/25
50000/50000 [==============================] - 16s - loss: 1.0694 - acc: 0.6249 - val_loss: 0.9963 - val_acc: 0.6485
Epoch 8/25
50000/50000 [==============================

In [15]:
model.save_weights('results/cifar10.h5')

In [14]:
model.load_weights('results/cifar10.h5')

### Precompute convolution output

In [16]:
layer_type = Convolution2D
layers = model.layers
layer_idx = [index for index,layer in enumerate(layers) if type(layer) is layer_type][-1]

In [17]:
conv_layers = layers[:layer_idx+1]
fc_layers = layers[layer_idx+1:]

In [18]:
conv_model = Sequential(conv_layers)

In [19]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 34, 34)     0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    896         zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 16, 16)    0           convolution2d_1[0][0]            
                                                                   convolution2d_1[1][0]            
___________________________________________________________________________________________

In [20]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]    

In [21]:
conv_feat = conv_model.predict(x_train)

In [22]:
save_array('results/cifar10_conv_feat.dat', conv_feat)

In [23]:
conv_val_feat = conv_model.predict(x_test)

In [24]:
save_array('results/cifar10_conv_val_feat.dat', conv_val_feat)

In [20]:
conv_feat = load_array('results/cifar10_conv_feat.dat')
conv_val_feat = load_array('results/cifar10_conv_val_feat.dat')

In [25]:
conv_layers[-1].output_shape

(None, 64, 14, 14)

### FC only

In [26]:
def get_fc_model(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]), # first layer must have input_shape
        Dropout(p / 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(p),
        Dense(10, activation='softmax')        
    ]

#### Learning rate = 0.0001, Dropout = 0.5

In [27]:
fc_model = Sequential(get_fc_model(0.5))
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
fc_model.fit(conv_feat, y_train, batch_size=batch_size, nb_epoch=epochs, validation_data=(conv_val_feat, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 4s - loss: 1.4560 - acc: 0.5140 - val_loss: 1.0381 - val_acc: 0.6582
Epoch 2/25
50000/50000 [==============================] - 4s - loss: 1.0207 - acc: 0.6478 - val_loss: 0.9104 - val_acc: 0.6851
Epoch 3/25
50000/50000 [==============================] - 4s - loss: 0.9220 - acc: 0.6795 - val_loss: 0.8562 - val_acc: 0.7051
Epoch 4/25
50000/50000 [==============================] - 4s - loss: 0.8654 - acc: 0.6985 - val_loss: 0.8289 - val_acc: 0.7137
Epoch 5/25
50000/50000 [==============================] - 4s - loss: 0.8308 - acc: 0.7138 - val_loss: 0.8117 - val_acc: 0.7191
Epoch 6/25
50000/50000 [==============================] - 4s - loss: 0.8082 - acc: 0.7203 - val_loss: 0.7974 - val_acc: 0.7239
Epoch 7/25
50000/50000 [==============================] - 4s - loss: 0.7837 - acc: 0.7286 - val_loss: 0.7844 - val_acc: 0.7258
Epoch 8/25
50000/50000 [==============================] - 4s 

## Add Batch normalization

In [29]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1), 
        Dropout(p / 2),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')        
    ]

#### Learning rate = 0.0001, Dropout = 0.5

In [30]:
bn_model = Sequential(get_bn_layers(0.5))
bn_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
bn_model.fit(conv_feat, y_train, batch_size=batch_size, nb_epoch=epochs, validation_data=(conv_val_feat, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 5s - loss: 1.4448 - acc: 0.5632 - val_loss: 0.8621 - val_acc: 0.7078
Epoch 2/25
50000/50000 [==============================] - 5s - loss: 1.0130 - acc: 0.6737 - val_loss: 0.7924 - val_acc: 0.7321
Epoch 3/25
50000/50000 [==============================] - 5s - loss: 0.8821 - acc: 0.7068 - val_loss: 0.7716 - val_acc: 0.7371
Epoch 4/25
50000/50000 [==============================] - 5s - loss: 0.8115 - acc: 0.7264 - val_loss: 0.7568 - val_acc: 0.7424
Epoch 5/25
50000/50000 [==============================] - 5s - loss: 0.7527 - acc: 0.7437 - val_loss: 0.7484 - val_acc: 0.7473
Epoch 6/25
50000/50000 [==============================] - 5s - loss: 0.7138 - acc: 0.7554 - val_loss: 0.7455 - val_acc: 0.7488
Epoch 7/25
50000/50000 [==============================] - 5s - loss: 0.6885 - acc: 0.7663 - val_loss: 0.7411 - val_acc: 0.7495
Epoch 8/25
50000/50000 [==============================] - 5s 

| | loss | acc | val_loss | val_acc |
|---|---|---|---|---|
| lr=0.0001, dropout=0.5 | 0.6768 | 0.7764 | 0.7716 | 0.7444 |
| BN, lr=0.0001, dropout=0.5 | 0.4751 | 0.8418 | 0.7227 | 0.7636 |